In [1]:
import pandas as pd
import numpy as np
import os
from typing import Callable # this is to define test_func functions dtype as function itself
import re # for regural expression
import warnings
warnings.filterwarnings('ignore')
import pickle # for pickling my model

In [2]:
df = pd.read_csv('mobile_recommendation_system_dataset.csv',index_col=False)

In [3]:
def random_values(col_name:str,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

In [4]:
random_values('corpus')

1336 : Storage32 GBRAM2 GBExpandable Storage256GB  SystemAndroid Pie 9.0Processor TypeMediaTek P22 Octa Core 2.0 GHzProcessor Speed2 13MP 2MP 5MP 3G Capacity4000 Display Size15.49 cm (6.1 inch)Resolution1560 x 720 pixelsResolution TypeHD+GPUGE8320Display Colors16.7MOther Display FeaturesScreen Texture: a-Si, In-cell Touch Panel Technology, Screen Ratio: 19.5:9, Touch Panel Glass Type: GG3, Screen Contrast: 1200/900, Color Saturation: 70%/65% NTSC, Maximum Brightness: 450 nits/400 nits
1429 : Storage128 GBRAM6 GBExpandable Storage512GB  SystemAndroid 12Processor TypeHelio G99Processor Speed2.2 50MP 2MP 2MP 50MP 2MP 2MP 50MP 8MP 8MP 4G Capacity5000 Display Size16.71 cm (6.58 inch)Resolution2408 x 1080 PixelsResolution TypeFull HD+GPUARM Mali-G57 MC2Display TypeFull HD+ LCD DisplayOther Display Features90Hz Refresh Rate, 400nits Peak Brightness
2126 : Storage64 GBRAM4 GBExpandable Storage256GB  SystemAndroid 10Processor TypeMediaTek Helio G35Processor Speed2.3 13MP 13MP 8MP 8MP 4G Capacit

## Lowercasing the corpus values

In [5]:
df['corpus'] = df['corpus'].str.lower()

In [6]:
random_values('corpus')

1432 : storage128 gbram4  systemandroid q 10processor typequalcomm snapdragon 662processor speed2 48mp 8mp 2mp 2mp 48mp 8mp 4g capacity6000 display size16.59 cm (6.53 inch)resolution2340 x 1080$$pixelresolution typefull hd+gpuadreno 610display typeips lcd
267 : storage64 gbram4 gbexpandable storage1tb  systemandroid 12processor typeunisoc sc9863a/ unisoc sc9863a1processor speed1.6 8mp 8mp 5mp 5mp 4g capacity5000 display size16.51 cm (6.5 inch)resolution1600 x 720 pixelsresolution typehd+gpuimagination ge8322display typehd+ lcd displaydisplay colors16.7mother display featuresrefresh rate: 60 hz, aspect ratio: 20:9, screen-to-body ratio: 88.7%, screen contrast: 1500:1, maximum brightness: 400 nits, color saturation: 70%, sunlight screen support
1933 : storage2 mbram10  systemandroid 10processor typeexynos octa coreprocessor speed2.73 64mp 12mp 12mp 64mp 12mp 12mp 10mp 10mp 4g capacity4300 display size17.02 cm (6.7 inch)resolution2400 x 1080 pixelsresolution typefull hd+display typefull h

In [7]:
df.isnull().sum()

name        0
ratings     0
price       0
imgURL      0
corpus     12
dtype: int64

In [8]:
df.dropna(inplace=True)

In [9]:
max_len = 0
for i in range(0, len(df['corpus'])):
    try:
        if len(df['corpus'][i]) > max_len:
            max_len = len(df['corpus'][i])
    except:
        pass

print(f'words max len in corpus is: {max_len}')


words max len in corpus is: 1196


In [10]:
df.sample(10)

,name,ratings,price,imgURL,corpus
1643,"MOTOROLA g52 (Metallic White, 128 GB)",4.2,13999,https://rukminim2.flixcart.com/image/312/312/l...,storage128 gbram6 systemandroid 12processor t...
2401,"Infinix Smart 2 (Serene Gold, 16 GB)",4.1,6999,https://rukminim2.flixcart.com/image/312/312/j...,storage16 gbram2 gbexpandable storage128gb sy...
1649,"SAMSUNG Galaxy A21s (Black, 128 GB)",4.0,16999,https://rukminim2.flixcart.com/image/312/312/k...,storage128 gbram6 gbexpandable storage512gb s...
1082,"Redmi 6 (Gold, 32 GB)",4.4,8999,https://rukminim2.flixcart.com/image/312/312/j...,storage32 gbram3 gbexpandable storage256gb sy...
662,"realme GT 2 (Paper White, 128 GB)",4.3,30537,https://rukminim2.flixcart.com/image/312/312/l...,storage128 gbram8 systemandroid 12processor t...
2478,"OPPO Reno3 Pro (Sky White, 256 GB)",4.4,"₹35,990",https://rukminim2.flixcart.com/image/312/312/k...,storage256 gbram8 gbexpandable storage256gb s...
2060,"REDMI Note 10S (Frost White, 64 GB)",4.4,16999,https://rukminim2.flixcart.com/image/312/312/k...,storage64 gbram6 gbexpandable storage512gb sy...
1010,"OnePlus 9RT 5G (Hacker Black, 128 GB)",3.9,22537,https://rukminim2.flixcart.com/image/312/312/x...,storage128 gbram8 systemandroid q 11processor...
1111,"realme C30s (Stripe Black, 64 GB)",4.2,8499,https://rukminim2.flixcart.com/image/312/312/x...,storage64 gbram4 gbexpandable storage1tb syst...
2265,"Tecno Spark Power (Dawn Blue, 64 GB)",4.2,10499,https://rukminim2.flixcart.com/image/312/312/k...,storage64 gbram4 gbexpandable storage256gb sy...


In [11]:
random_values('imgURL')

1352 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/q/g/a/-original-imagndzzegs4guek.jpeg?q=70
1606 : https://rukminim2.flixcart.com/image/312/312/kk2wl8w0/mobile/i/t/a/m02s-sb08rswhmz61-samsung-original-imafzg4azbgz4c3y.jpeg?q=70
796 : https://rukminim2.flixcart.com/image/312/312/k9stjm80/mobile/8/q/9/vivo-y91i-vivo-1820-pd1818hf-ex-original-imafrgnrngnjwfka.jpeg?q=70
1965 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/p/w/b/-original-imagmefcaj26vdhg.jpeg?q=70
440 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/z/c/0/spark-go-2023-bf7-tecno-original-imagzg8f73aycpee.jpeg?q=70
1156 : https://rukminim2.flixcart.com/image/312/312/xif0q/mobile/u/1/h/-original-imagpfbvfu4p55n4.jpeg?q=70


# Model

## vectorization(to convert the words to vectors)

In [12]:

from sklearn.feature_extraction.text import CountVectorizer 

In [13]:
cont_vect = CountVectorizer(max_features=1000)

In [14]:
arrays = cont_vect.fit_transform(df['corpus'])

In [15]:
vectors = arrays.toarray()

In [16]:
cont_vect.get_feature_names_out()

array(['00', '000', '02', '03', '04', '05', '07', '07bother', '08', '09',
       '0mp', '0primary', '0processor', '10', '100', '1000', '100000',
       '1000000', '100mp', '103', '104', '106', '1080', '1080processor',
       '1080resolution', '108mp', '10mp', '10processor', '11', '1100',
       '1125', '1170', '1179', '11primary', '11processor', '12', '120',
       '1200', '1200processor', '120hz', '1242', '1280processor', '1284',
       '1290', '12mp', '12primary', '12processor', '13', '1300',
       '1300processor', '1330', '1334', '135', '13mp', '13processor',
       '14', '1400', '144', '1440', '144hz', '1480', '14processor', '15',
       '1500', '1544', '15processor', '16', '160', '1600', '16000',
       '1600resolution', '1612', '16mp', '16processor', '18', '180',
       '1800', '180hz', '19', '1920', '1920hz', '1processor', '20', '200',
       '2000000', '200mp', '20mp', '21', '22', '23', '2340', '24', '240',
       '2400', '2400resolution', '2408', '240hz', '2460', '24mp', '25m

## Calculating cosine similarity for recommendation

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
similarity = cosine_similarity(vectors)
similarity.shape # this is because I have calculated the similarity among all the rows to all the rows(confusing?? but try to understand)

(2534, 2534)

In [19]:
similarity[3]

array([0.3345748 , 0.17485208, 0.53136893, ..., 0.57564968, 0.27586342,
       0.37283655])

In [20]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:11]

[(68, 0.9999999999999998),
 (303, 0.9999999999999998),
 (616, 0.9850746268656714),
 (1461, 0.9850746268656714),
 (457, 0.970149253731343),
 (1027, 0.970149253731343),
 (1735, 0.970149253731343),
 (569, 0.9468131937613656),
 (1123, 0.9468131937613656),
 (1540, 0.9468131937613656)]

In [21]:
def recommend(mobile):
    mobile_index = df[df['name']==mobile].index[0]
    similarity_array = similarity[mobile_index]
    similar_10_mobiles = sorted(list(enumerate(similarity_array)),reverse=True,key=lambda x:x[1])[1:11]
    
    for i in similar_10_mobiles:
        print(df['name'].iloc[i[0]])



In [22]:
recommend('APPLE iPhone 13 mini (Blue, 128 GB)')

APPLE iPhone 13 mini ((PRODUCT)RED, 128 GB)
APPLE iPhone 13 mini (Starlight, 128 GB)
APPLE iPhone 13 mini (Green, 128 GB)
APPLE iPhone 13 mini (Midnight, 128 GB)
APPLE iPhone 13 mini (Green, 256 GB)
APPLE iPhone 13 mini (Midnight, 512 GB)
APPLE iPhone 13 mini (Pink, 512 GB)
APPLE iPhone 13 mini (Starlight, 256 GB)
APPLE iPhone 13 mini (Midnight, 256 GB)
APPLE iPhone 13 mini (Pink, 256 GB)


## making Pickle file of df and similarities to use in web

In [ ]:
#os.makedirs(r'src\model',exist_ok=True)

In [23]:
pickle.dump(obj=df,file=open(file=r'dataframe.pkl',mode='wb'))

In [24]:
pickle.dump(obj=similarity,file=open(file=r'similarity.pkl',mode='wb'))